In [2]:
import re

In [57]:
from ctypes import * 
libFile = r'F:\others\applications\nlpir_parser\bin\NLPIR.dll'
dll = CDLL(libFile)

In [58]:
def loadFun(exportName, restype, argtypes):
    global dll
    f = getattr(dll,exportName)
    f.restype = restype
    f.argtypes = argtypes
    return f

class ENCODING:
    GBK_CODE        =   0               #默认支持GBK编码
    UTF8_CODE       =   GBK_CODE+1      #UTF8编码
    BIG5_CODE       =   GBK_CODE+2      #BIG5编码
    GBK_FANTI_CODE  =   GBK_CODE+3      #GBK编码，里面包含繁体字
 
class SegAtom(Structure):
    _fields_ = [("start", c_int32), ("length", c_int32),
        ("sPOS", c_char * 40),      ("iPOS", c_int32),
        ("word_ID", c_int32),       ("word_type", c_int32), ("weight", c_int32)
    ]
 
Init = loadFun('NLPIR_Init',c_int, [c_char_p, c_int, c_char_p])
ParagraphProcessA = loadFun('NLPIR_ParagraphProcessA',POINTER(SegAtom), [c_char_p, c_void_p, c_bool])

if not Init(b'',ENCODING.UTF8_CODE,b''):
    print("Initialization failed!")
    exit(-111111) 
    
def segment(paragraph):
    count = c_int32()
    result = ParagraphProcessA(paragraph, byref(count),c_bool(True))
    count = count.value
    atoms = cast(result, POINTER(SegAtom))
    return [atoms[i] for i in range(0,count)]
 
def Seg(paragraph):
    atoms = segment(paragraph) #调用segment()
    for a in atoms:
        if len(a.sPOS) < 1: continue
        i = paragraph[a.start: a.start + a.length]#.decode('utf-8')#.encode('type')
        yield (i, a.sPOS)

Initialization failed!


In [49]:
str = '此篇论文以单个学者的写作风格在50年之内是否发生变化为研究问题，通过追踪以往研究的方式从纵向角度探究写作风格是否发生了变化'
segment(str)

ArgumentError: argument 1: <class 'TypeError'>: wrong type

In [44]:
f = codecs.open(r'E:/Ipython/Machine Learning/research/corpus_by_gene/30.txt', 'rb', "utf-8")
p = f.read().encode('utf-8')  # 读入txt文件的所有内容
for t in Seg(p):
    print(t[0],t[1])

In [6]:
path = r'E:/Ipython/Machine Learning/research/corpus_by_gene/'

In [3]:
def parse_file(target_path):
    """to read file from file paths"""
    with open(target_path, 'rb') as file:
        content = file.read()
        text = content.decode('utf-8')
        # to delete /n and /w
        file.close()
    return text

In [4]:
def remove_elements(sentences, ele):
    txt = sentences
    text = re.sub(ele, '', txt)
    return text
def remove_punc(sentences):
    punct = set(u''':!),.:;?]}¢'"〉=>/》」』】〕〗〞︰︱︳﹐､﹒`
    ﹔﹕﹖﹗﹚﹜﹞）．｜｝︴︶︸︺︼︾﹀﹂﹄﹏､～￠
    々‖•·ˇˉ―--′’”([{£¥'"‵〈《「『【〔〖（［｛￡￥〝︵︷︹︻
    ︽︿﹁﹃﹙﹛﹝（｛“‘-—_…''')
    # for str/unicode
    filter_punt = lambda s: ''.join(filter(lambda x: x not in punct, s))
    # for list
    filter_punt_l = lambda l: list(filter(lambda x: x not in punct, l))
    if not isinstance(sentences, list):
        filter_text = filter_punt(sentences)
    else:
        filter_text = filter_punt_l(sentences)
    pure_text = re.sub(u'\u3000',u'',filter_text)
    return pure_text

def remove_non_chinese(sentence):
    """to remove non chinese character"""
    content = [sentence]
    chinese_list = []
    for line in content:
        chinese_list.append(format_str(line))
    return chinese_list[0]


def is_chinese(uchar):
    """汉字编码区间"""
    if u'\u4e00' <= uchar <= u'\u9fa5':
        return True
    else:
        return False


def format_str(content):
    """to recognize chinese character"""
    content_str = ""
    for i in content:
        if is_chinese(i):
            content_str = content_str + i
    return content_str

In [7]:
content = parse_file(path+'30.txt')

In [8]:
text = re.sub(',','，', content)
a = remove_punc(text)
pure_text = a
useless_char = ['\n', '\t', ' ','\r']
for char in useless_char:
    pure_text = remove_elements(pure_text, char)

In [9]:
te = re.sub(u'\u3000',u'', pure_text)
te

'今年，是我国著名的图书馆学家、文献分类学家、图书馆学教育家和近代图书馆事业奠基者之一的杜定友先生18981967诞辰100周年。为了纪念这位先驱者，我认为很有必要研究他的精神，让他的精神永存，以指导我国图书馆事业不断向前发展，更好地为人民服务。杜定友先生的精神，概括起来有8点，即敬业献身精神、刻苦钻研精神、开拓进取精神、艰苦奋斗精神、讲究实效精神、读者至上精神、尊师重教精神、爱国主义精神。现分别阐述如下。1敬业献身精神杜定友先生1898年1月7日出生于上海市。原籍广东省南海县西樵乡大果村人。祖父是个皮鞋匠，因为家贫，便携带妻子儿女往上海谋生。父亲杜顺荣在上海学会了照相，开了一间小照相馆。杜先生1919年在上海工业专门学校南洋大学的前身，后改交通大学附属中学毕业时，由于学习成绩优良，获得奖励，被学校破格保送到菲律宾大学图书馆学系学习。1920年被授予菲律宾大学文学士学位，1921年毕业时又被授予教育学和图书馆学两个学士学位。杜定友先生在读书时，就已对图书馆工作产生兴趣，图书馆学系毕业后更认识到办图书馆可以保存文献，提倡学术，普及教育，提高文化，对于社会国家，有很大意义。对于人类，有很大贡献。1所以回国后，就积极宣传办图书馆的意义和作用，宣传欧美图书馆事业和图书馆技术方法，提倡发展中国图书馆事业，把新图书馆运动推向了高潮。并身体力行，终身为图书馆事业而奋斗。杜定友先生在图书馆工作中，不求名利，只讲奉献，体现出了高尚的、宝贵的敬业献身精神，达到了社会道德的最高境界，其突出表现是第一，把从事图书馆工作视为无尚的光荣和莫大的幸福。他在我和图书馆一文中，曾自认为与图书馆结婚，形影不离，爱之弥深，敬之弥笃。并充满了自豪感，把从事图书馆工作看作胜过做大总统。第二，自觉自愿地为图书馆事业献身。杜先生曾说我愿做图书馆界的黄花岗烈士。一个人为了一种事业，如果不能牺牲个人，则只有牺牲事业。为了图书馆事业，我都牺牲了。但我所希望的成果，我并没有看到，好象黄花岗烈士，并没有看到民国的出现。他日社会改革，政治上轨道，经济建设完成，图书馆事业的发扬光大，会当有日，成功不自我，鞠躬尽瘁，死而无憾第三，竭尽全力为图书馆事业服务。他在我与图书馆学一文中说我对于图书馆的宣传却尽了一点力量，我自问还能跑能讲能写。奔走四方，大声疾呼，有时还带了图表幻灯影片，到处宣传，宣传对象往往是广大的民众。即发表文字，虽

In [52]:
sencences = te.split('。')
len(sencences) 

5728

In [56]:
from nlpir import * 
SetPOSmap(2)
#for secence in sencences:
p = sencences[1]
p = p.encode('UTF8')
s = []
for t in Seg(p):
    s.append(t[1])
print(s)

['p', 'v', 'r', 'q', 'n', 'w', 'r', 'v', 'd', 'v', 'a', 'vn', 'r', 'u', 'n', 'w', 'v', 'r', 'u', 'n', 'd', 'v', 'w', 'p', 'vn', 'n', 'n', 'n', 'd', 'v', 'v', 'w', 'd', 'a', 'u', 'v', 'n', 'vn']


In [77]:

def pos_statistic(pos_li, t_pos,c):
    """to count target pos in every sentence"""
    c=c
    for p in pos_li:
        for j in t_pos:
            if p == j:
                c += 1
            return c

v_pos = ['v', 'vg', 'vd', 'vn']
vc = pos_statistic(v_pos, s,0)
vc

0

11

In [34]:
# 计算分词后每句话词的数量（每句话的长度）
# 计算每句话的词性占比计算
# 计算非停用词数量占比+学术用词占比
import pandas as pd


In [37]:
df = pd.DataFrame(index = ("2",3), columns = ('i','you'))

In [39]:
a = '30'
b = 1

In [47]:
print(('I am %d ears old %s')%(b,a))

I am 1 ears old 30


In [48]:
sum([0,0,0,0,2])

2